In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [105]:
def scrape_all_players():
    
    url = "https://www.basketball-reference.com/players/"
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    data = pd.DataFrame()
    player_name = []
    career_length = []
    seasons_played = []
    gp = []
    winshare = []
    mppg = []
    ppg = []
    apg = []
    rpg = []
    playoff_mp = []
    playoff_ppg = []
    playoff_apg = []
    playoff_rpg = []
    total_pts = []
    total_ast = []
    total_reb = []
    fg_percentage = []
    three_percentage = []
    ft_percentage = []
    per = []
    rings = []
    mvps = []
    all_star_appearances = []
    dpoy = []
    plus_minus = []
    hof = []

    for i in range(len(alphabet)):

        url_ln = url + alphabet[i]
        r = requests.get(url_ln)
        soup = BeautifulSoup(r.text, 'html.parser')

        num_players = int(soup.find(id='players').findChildren()[0].text.split(' ')[0])
        players = soup.find(id='players').findAll('tr')
        
        for j in range(1, len(players)):
            
            name = players[j].findAll('a')[0].text
            name_split = name.lower().replace(" jr.", '').replace("'", '').split(' ')

            print(name_split)
            if(len(name_split) >= 2):

                player_name.append(name)
                start_yr = int(players[j].findAll('td', {'data-stat': 'year_min'})[0].text)
                end_yr = int(players[j].findAll('td', {'data-stat': 'year_max'})[0].text)
                career_length.append(end_yr - start_yr)

                if len(name_split[1]) < 5:
                    if len(name_split[0]) < 2:
                        url_player = url_ln + "/" + name_split[-1] + name_split[0] + "01.html"
                    else:
                        url_player = url_ln + "/" + name_split[-1] + name_split[0][:2] + "01.html"
                else: 
                    if len(name_split[0]) < 2:
                        url_player = url_ln + "/" + name_split[-1][:5] + name_split[0] + "01.html"
                    else: 
                        url_player = url_ln + "/" + name_split[-1][:5] + name_split[0][:2] + "01.html"
            
                player_r = requests.get(url_player)
                player_soup = BeautifulSoup(player_r.text, 'html.parser')

                if len(player_soup.findAll('span', {'data-tip':'Games'})) > 0:
                    gp.append(player_soup.findAll('span', {'data-tip':'Games'})[0].findNextSiblings()[1].text)
                else:
                    gp.append(-1)
                if len(player_soup.findAll('span', {'data-tip':'Points'})) > 0:
                    ppg.append(player_soup.findAll('span', {'data-tip':'Points'})[0].findNextSiblings()[1].text)
                else:
                    ppg.append(-1)
                if len(player_soup.findAll('span', {'data-tip':'Total Rebounds'})) > 0:
                    rpg.append(player_soup.findAll('span', {'data-tip':'Total Rebounds'})[0].findNextSiblings()[1].text)
                else:
                    rpg.append(-1)
                if len(player_soup.findAll('span', {'data-tip':'Assists'})) > 0:
                    apg.append(player_soup.findAll('span', {'data-tip':'Assists'})[0].findNextSiblings()[1].text)
                else:
                    apg.append(-1)
                if len(player_soup.findAll('span', {'data-tip':'Field Goal Percentage'})) > 0:
                    fg_percentage.append(player_soup.findAll('span', {'data-tip':'Field Goal Percentage'})[0].findNextSiblings()[1].text)
                else:
                    fg_percentage.append(-1)
                if len(player_soup.findAll('span', {'data-tip':'3-Point Field Goal Percentage'})) > 0:
                    three_percentage.append(player_soup.findAll('span', {'data-tip':'3-Point Field Goal Percentage'})[0].findNextSiblings()[1].text)
                else:
                    three_percentage.append(-1)
                if len(player_soup.findAll('span', {'data-tip':'Free Throw Percentage'})) > 0:
                    ft_percentage.append(player_soup.findAll('span', {'data-tip':'Free Throw Percentage'})[0].findNextSiblings()[1].text)
                else:
                    ft_percentage.append(-1)
                if len(player_soup.findAll('span', {'data-tip':'<b>Player Efficiency Rating</b><br>A measure of per-minute production standardized such that the league average is 15.'})) > 0:
                    per.append(player_soup.findAll('span', {'data-tip':'<b>Player Efficiency Rating</b><br>A measure of per-minute production standardized such that the league average is 15.'})[0].findNextSiblings()[1].text)
                else:
                    per.append(-1)
                if len(player_soup.findAll('span', {'data-tip':'<b>Win Shares</b><br>An estimate of the number of wins contributed by a player.'})) > 0:
                    winshare.append(player_soup.findAll('span', {'data-tip':'<b>Win Shares</b><br>An estimate of the number of wins contributed by a player.'})[0].findNextSiblings()[1].text)
                else:
                    winshare.append(-1)
            



    data['Player'] = player_name
    data['Career Length'] = career_length
    data['GP'] = gp
    data['PPG'] = ppg
    data['APG'] = apg
    data['RPG'] = rpg
    data['FG%'] = fg_percentage
    data['3P%'] = three_percentage
    data['FT%'] = ft_percentage
    data['PER'] = per
    data['Winshares'] = winshare

    return data

            
            
            
            


data = scrape_all_players()
data

['alaa', 'abdelnaby']
['zaid', 'abdul-aziz']
['kareem', 'abdul-jabbar']
['mahmoud', 'abdul-rauf']
['tariq', 'abdul-wahad']
['shareef', 'abdur-rahim']
['tom', 'abernethy']
['forest', 'able']
['john', 'abramovic']
['álex', 'abrines']
['precious', 'achiuwa']
['alex', 'acker']
['don', 'ackerman']
['mark', 'acres']
['bud', 'acton']
['quincy', 'acy']
['alvan', 'adams']
['don', 'adams']
['george', 'adams']
['hassan', 'adams']
['jaylen', 'adams']
['jordan', 'adams']
['michael', 'adams']
['steven', 'adams']
['rafael', 'addison']
['bam', 'adebayo']
['deng', 'adel']
['rick', 'adelman']
['jeff', 'adrien']
['arron', 'afflalo']
['maurice', 'ager']
['mark', 'aguirre']
['blake', 'ahearn']
['danny', 'ainge']
['matthew', 'aitch']
['alexis', 'ajinça']
['henry', 'akin']
['josh', 'akognon']
['devaughn', 'akoon-purcell']
['solomon', 'alabi']
['mark', 'alarie']
['gary', 'alcorn']
['santi', 'aldama']
['furkan', 'aldemir']
['cole', 'aldrich']
['lamarcus', 'aldridge']
['chuck', 'aleksinas']
['cliff', 'alexander

,Player,Career Length,GP,PPG,APG,RPG,FG%,3P%,FT%,PER,Winshares
0,Alaa Abdelnaby,4,256,5.7,0.3,3.3,50.2,0.0,70.1,13.0,4.8
1,Zaid Abdul-Aziz,9,505,9.0,1.2,8.0,42.8,-1,72.8,15.1,17.5
2,Kareem Abdul-Jabbar,19,1560,24.6,3.6,11.2,55.9,5.6,72.1,24.6,273.4
3,Mahmoud Abdul-Rauf,10,724,12.6,4.9,3.0,44.1,-1,75.7,15.0,30.3
4,Tariq Abdul-Wahad,5,236,7.8,1.1,3.3,41.7,23.7,70.3,11.4,3.5
...,...,...,...,...,...,...,...,...,...,...,...
5017,Ante Žižić,2,-1,-1,-1,-1,-1,-1,-1,-1,-1
5018,Jim Zoet,0,7,0.3,0.1,1.1,20.0,-,-,-0.8,-0.1
5019,Bill Zopf,0,53,2.2,1.4,0.9,36.3,-1,55.6,9.6,-0.1
5020,Ivica Zubac,5,360,8.3,1.1,6.5,59.7,10.0,75.4,19.2,26.1


In [ ]:
data.to_csv('./all_player_data.csv')